In [211]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from sklearn.feature_selection import mutual_info_classif
from keras.layers import LeakyReLU
from sklearn.preprocessing import LabelEncoder

# DOI 10.24432/C5HP4Z

In [212]:
nomes_colunas = ['ID',  'Diagnosis', 'radius1', 'texture1', 'perimeter1', 'area1', 'smoothness1',
       'compactness1', 'concavity1', 'concave_points1', 'symmetry1',
       'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 'area2',
       'smoothness2', 'compactness2', 'concavity2', 'concave_points2',
       'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3',
       'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3',
       'symmetry3', 'fractal_dimension3']

In [213]:
df = pd.read_csv('wdbc.data')
df_original = df
df.columns = nomes_colunas

In [214]:
df.head()

,ID,Diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
2,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
3,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
4,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  568 non-null    int64  
 1   Diagnosis           568 non-null    object 
 2   radius1             568 non-null    float64
 3   texture1            568 non-null    float64
 4   perimeter1          568 non-null    float64
 5   area1               568 non-null    float64
 6   smoothness1         568 non-null    float64
 7   compactness1        568 non-null    float64
 8   concavity1          568 non-null    float64
 9   concave_points1     568 non-null    float64
 10  symmetry1           568 non-null    float64
 11  fractal_dimension1  568 non-null    float64
 12  radius2             568 non-null    float64
 13  texture2            568 non-null    float64
 14  perimeter2          568 non-null    float64
 15  area2               568 non-null    float64
 16  smoothne

In [216]:
df['Diagnosis'].value_counts()

Diagnosis
B    357
M    211
Name: count, dtype: int64

In [217]:
df.drop(columns=['ID'], inplace=True)
#df['Diagnosis'] = df['Diagnosis'].replace({'M': 4, 'B': 2})

le = LabelEncoder()
df['Diagnosis'] = le.fit_transform(df['Diagnosis'])

df.replace('?', np.nan, inplace=True)
df = df.astype('float64')
df.dropna(inplace=True)

In [218]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import random

In [219]:
scaler = MinMaxScaler()
df = scaler.fit_transform(df)

In [220]:
melhor_k = 0
melhor_razao = 0
melhor_wss = 0
melhor_bss = 0
seed_salva = 0

for k in range(1, 8):
    for i in range(1, 300):
        seed = random.randint(0, 100000)
        kmeans = KMeans(n_clusters=k, random_state=seed)
        kmeans.fit(df)
        
        wss = kmeans.inertia_
        total_ss = np.sum((df - np.mean(df, axis=0))**2).sum()
        bss = total_ss - wss
        razao = bss/total_ss
    
        if razao > melhor_razao:
            melhor_razao = razao
            melhor_k = k
            melhor_wss = wss
            melhor_bss = bss
            melhor_seed = seed
        
        print(f"razao: {razao:0.5f} | k: {k} | seed: {seed}", end='\r')

print(f"WSS (Within-Cluster Sum of Squares): {melhor_wss}")
print(f"BSS (Between-Cluster Sum of Squares): {melhor_bss}")
print(f"Melhor razao BSS/(WSS + BSS): {melhor_razao} | melhor k: {melhor_k} | seed {melhor_seed}")

WSS (Within-Cluster Sum of Squares): 142.3056016417273
BSS (Between-Cluster Sum of Squares): 342.1556543413874
Melhor razao BSS/(WSS + BSS): 0.7062600984407983 | melhor k: 7 | seed 25614


In [221]:
def information_gain(X, y):
    info_gains = {}
    
    # Entropia do alvo (Diagnosis)
    entropy_y = -np.sum(np.log2(y.value_counts(normalize=True)) * y.value_counts(normalize=True))
    
    # Calcular Information Gain para cada feature
    for feature in X.columns:
        # Calcular a entropia condicional (com base na feature)
        feature_values = X[feature]
        
        # Média ponderada das entropias das partes (conjuntos distintos)
        feature_entropy = 0
        for value in feature_values.unique():
            # Dividir os dados em subconjuntos para cada valor único da feature
            subset_y = y[feature_values == value]
            subset_entropy = -np.sum(np.log2(subset_y.value_counts(normalize=True)) * subset_y.value_counts(normalize=True))
            weight = len(subset_y) / len(y)
            feature_entropy += weight * subset_entropy
        
        # Calcular o Information Gain para a feature
        info_gain = entropy_y - feature_entropy
        info_gains[feature] = info_gain
    
    return info_gains

In [229]:
kmeans = KMeans(n_clusters=melhor_k, random_state=melhor_seed)
kmeans.fit(df)

df = pd.DataFrame(df, columns=df_original.columns)
df['cluster'] = kmeans.fit_predict(df)

X = df.drop(columns=['Diagnosis', 'cluster'])
y = df['Diagnosis']

info_gains = information_gain(X, y)
info_gains = pd.Series(info_gains).sort_values(ascending=False)

In [230]:
info_gains

concave_points1       0.941237
smoothness2           0.934195
concavity1            0.934195
area3                 0.934195
radius2               0.932866
perimeter2            0.930674
concavity3            0.929345
area1                 0.927153
concavity2            0.927153
perimeter1            0.925824
area2                 0.924495
compactness2          0.922303
compactness3          0.917453
fractal_dimension2    0.916590
concave_points3       0.913932
compactness1          0.908218
radius3               0.899381
perimeter3            0.897655
fractal_dimension3    0.890613
texture2              0.863307
texture3              0.862444
radius1               0.859786
concave_points2       0.857593
symmetry3             0.844372
fractal_dimension1    0.835138
texture1              0.834672
symmetry2             0.817066
smoothness1           0.775034
symmetry1             0.733819
smoothness3           0.718475
dtype: float64